In [1]:
import tensorflow as tf
import os
import pickle
import numpy as np
DataSet = "model_input.npy"  # 数据集

In [2]:
def load_data(filename):
    """获取数据"""
    arr=np.load(filename,allow_pickle=True)
    arr1=[]
    arr2=[]
    for i in range(len(arr)):
        arr1.append(arr[i][0])
        arr2.append(arr[i][1])
    return arr1,arr2
    
class CifarData:
    """打乱数据集"""
    def __init__(self, filenames, need_shuffle):  # 训练集需要打乱
        all_data = []
        all_labels = []
        '''for filename in filenames:
            data, labels = load_data(filename)
            all_data.append(data)
            all_labels.append(labels)'''
        data,labels=load_data(filenames)
        all_data.append(data)
        all_labels.append(labels)
        
        self._data = np.vstack(all_data)  # 转为纵向矩阵,10000组，每组3072数
        self._data = np.reshape(self._data,(len(self._data),-1))  # 转为纵向矩阵,10000组，每组3072数
        # print(self._data)
        self._labels = np.hstack(all_labels)  # 转为横向矩阵，10000个数
        # print(self._data.shape)
        # print(self._labels.shape)
        
        self._num_examples = self._data.shape[0]  # 训练集总数量
        # print(self._num_examples)
        self._need_shuffle = need_shuffle
        self._indicator = 0  # 当前遍历数据集的位置
        if self._need_shuffle:  # 判断是否需要打乱数据
            self._shuffle_data()
            
    def _shuffle_data(self):  # 打乱数据
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
    
    def next_batch(self, batch_size):  # 数据分组，每次取不同的组
        """return batch_size examples as a batch."""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:  # 考察位置大于总数，重新打乱数据，重新分组
            if self._need_shuffle:  # 可以打乱
                self._shuffle_data()  # 重新打乱
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        if end_indicator > self._num_examples:  # 分块大小过大
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels
    
train_data=CifarData("train_file_1.npy",True)
test_data=CifarData("test_file_1.npy",False)

In [3]:
def conv_wrapper(inputs,name,output_channel=32,\
    kernel_size=(3,3),strides=1,activation=tf.nn.relu,kernel_initializer=tf.truncated_normal_initializer(stddev=0.02)):
    """卷积层装饰器"""
    # with batch normalization:conv->bn->activation
    with tf.name_scope(name):
        conv2d=tf.layers.conv2d(inputs,
                         output_channel,
                         kernel_size,
                         strides=strides,
                         padding='same',
                         activation=None,
                         kernel_initializer=kernel_initializer,
                         data_format='channels_first',
                         name=name+'/conv2d')
        bn=tf.layers.batch_normalization(conv2d,training=True)
        return activation(bn)
        
def pooling_wrapper(inputs,name):
    """池化层装饰器"""
    return tf.layers.max_pooling2d(inputs,(2,2),(2,2),name=name,padding='same',data_format='channels_first')

In [4]:
# 定义计算图
x = tf.placeholder(tf.float32, [None, 11264])  # 设置占位符
x_image=tf.reshape(x,[-1,1,88,128])  # 图像规模
x_image=tf.transpose(x_image,perm=[0,2,3,1])  # 通道转换
y = tf.placeholder(tf.int64, [None])  # y为的标注

#conv1
conv1_1=conv_wrapper(x_image,'conv1_1')
conv1_2=conv_wrapper(conv1_1,'conv1_2')
conv1_3=conv_wrapper(conv1_2,'conv1_3')
pooling1=pooling_wrapper(conv1_3,'pool1')
#conv2
conv2_1=conv_wrapper(pooling1,'conv2_1',output_channel=64)
conv2_2=conv_wrapper(conv2_1,'conv2_2',output_channel=64)
conv2_3=conv_wrapper(conv2_2,'conv2_3',output_channel=64)
pooling2=pooling_wrapper(conv2_3,'pool2')
#conv3
conv3_1=conv_wrapper(pooling2,'conv3_1',output_channel=64)
conv3_2=conv_wrapper(conv3_1,'conv3_2',output_channel=64)
conv3_3=conv_wrapper(conv3_2,'conv3_3',output_channel=64)
pooling3=pooling_wrapper(conv3_3,'pool3')

#flat(平坦化)
flatten=tf.layers.flatten(pooling3)  # 在保留第0轴的情况下对输入的张量进行Flatten(扁平化)
#输出 全连接层 输出形状[?,10]
logits=tf.layers.dense(flatten,10)  # 全连接层

loss=tf.losses.sparse_softmax_cross_entropy(labels=y,logits=logits)  # 交叉熵损失函数：y_->softmax,y->onhot,loss=ylogy_

predict = tf.argmax(logits,1)  # 样本中分布的最大值的位置，得到index
correct_prediction = tf.equal(predict, y)  # 相等为1，不相等为0
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))  # 平均数即为准确率

with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)  # adam优化器，定义梯度下降方法，使损失函数最小

In [5]:
# 执行计算图
init = tf.global_variables_initializer()  # 变量初始化
batch_size = 100  # 每次选取数据量
train_steps = 20000  # 迭代次数
test_steps = 2
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)  # 获取部分数据
        # print(batch_labels)
        # print(batch_data)
        loss_val, acc_val, _ = sess.run(
            [loss, accuracy, train_op],
            feed_dict={
                x: batch_data,
                y: batch_labels})  # 得到训练的损失值和准确率。对所有的参数计算梯度，然后用梯度去更新参数。
        if (i+1) % 100 == 0:
            print('Train-Step: %d, loss: %4.5f, acc: %4.5f' % (i+1, loss_val, acc_val))
        if (i+1) % 2000 == 0:  # 测试集验证
            test_data = CifarData("test_file_1.npy", False)  # 每次都重用测试集进行测试
            all_test_acc_val = []
            for j in range(test_steps): 
                test_batch_data, test_batch_labels \
                    = test_data.next_batch(batch_size)
                test_acc_val = sess.run(
                    [accuracy],
                    feed_dict = {
                        x: test_batch_data, 
                        y: test_batch_labels
                    })  # 模型检测测试集准确率
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('Test: %d, acc: %4.5f' % (i+1, test_acc))  # 计算测试集准确率
            saver.save(sess, "./VGGmodel/VGG"+str(i)+".ckpt")

Train-Step: 100, loss: 0.35810, acc: 0.90000
Train-Step: 200, loss: 0.40720, acc: 0.87000
Train-Step: 300, loss: 0.27852, acc: 0.91000
Train-Step: 400, loss: 0.23203, acc: 0.93000
Train-Step: 500, loss: 0.25536, acc: 0.90000
Train-Step: 600, loss: 0.48447, acc: 0.85000
Train-Step: 700, loss: 0.22241, acc: 0.91000
Train-Step: 800, loss: 0.52659, acc: 0.88000
Train-Step: 900, loss: 0.28135, acc: 0.92000
Train-Step: 1000, loss: 0.21112, acc: 0.93000
Train-Step: 1100, loss: 0.21257, acc: 0.93000
Train-Step: 1200, loss: 0.33927, acc: 0.93000
Train-Step: 1300, loss: 0.34803, acc: 0.87000
Train-Step: 1400, loss: 0.24197, acc: 0.92000
Train-Step: 1500, loss: 0.22259, acc: 0.92000
Train-Step: 1600, loss: 0.14843, acc: 0.95000
Train-Step: 1700, loss: 0.21237, acc: 0.91000
Train-Step: 1800, loss: 0.10804, acc: 0.96000
Train-Step: 1900, loss: 0.13491, acc: 0.95000
Train-Step: 2000, loss: 0.14829, acc: 0.96000
Test: 2000, acc: 0.97500
Train-Step: 2100, loss: 0.10564, acc: 0.96000
Train-Step: 2200, 

Train-Step: 17400, loss: 0.33456, acc: 0.87000
Train-Step: 17500, loss: 0.32392, acc: 0.86000
Train-Step: 17600, loss: 0.40147, acc: 0.87000
Train-Step: 17700, loss: 0.43120, acc: 0.84000
Train-Step: 17800, loss: 0.40971, acc: 0.83000
Train-Step: 17900, loss: 0.23102, acc: 0.92000
Train-Step: 18000, loss: 0.30740, acc: 0.91000
Test: 18000, acc: 0.51500
Train-Step: 18100, loss: 0.50038, acc: 0.81000
Train-Step: 18200, loss: 0.30162, acc: 0.91000
Train-Step: 18300, loss: 0.40962, acc: 0.87000
Train-Step: 18400, loss: 0.27289, acc: 0.88000
Train-Step: 18500, loss: 0.23104, acc: 0.93000
Train-Step: 18600, loss: 0.32493, acc: 0.89000
Train-Step: 18700, loss: 0.28293, acc: 0.92000
Train-Step: 18800, loss: 0.33070, acc: 0.87000
Train-Step: 18900, loss: 0.27750, acc: 0.93000
Train-Step: 19000, loss: 0.28272, acc: 0.88000
Train-Step: 19100, loss: 0.34103, acc: 0.85000
Train-Step: 19200, loss: 0.42033, acc: 0.86000
Train-Step: 19300, loss: 0.29224, acc: 0.89000
Train-Step: 19400, loss: 0.22730, 